<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/ComputerVision/DeepLearning/MiniGoogleNet_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNet - Rede Neural Convolucional - Classificação de Imagens - Cifar10

## Leitura de dados

In [1]:
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/minigooglenet.py
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/trainingmonitor.py

--2019-04-19 16:17:25--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/minigooglenet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3686 (3.6K) [text/plain]
Saving to: ‘minigooglenet.py’

minigooglenet.py    100%[===================>]   3.60K  --.-KB/s    in 0s      

2019-04-19 16:17:26 (47.2 MB/s) - ‘minigooglenet.py’ saved [3686/3686]

--2019-04-19 16:17:27--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/trainingmonitor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP

In [2]:
!ls -ila 

total 32
23199751 drwxr-xr-x 1 root root 4096 Apr 19 16:17 .
 5373954 drwxr-xr-x 1 root root 4096 Apr 19 16:16 ..
23199752 drwxr-xr-x 1 root root 4096 Apr  4 20:19 .config
 5637282 -rw-r--r-- 1 root root 3686 Apr 19 16:17 minigooglenet.py
 9306120 drwxr-xr-x 1 root root 4096 Apr  4 20:20 sample_data
 5637284 -rw-r--r-- 1 root root 1395 Apr 19 16:17 trainingmonitor.py


In [3]:
# Mini GoogleLeNet

# Imports
import os
import numpy as np
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from minigooglenet import MiniGoogLeNet
from trainingmonitor import TrainingMonitor
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator


modelo_path=''
output='' 


# Define o número total de épocas para treinar junto com a taxa inicial de aprendizado
NUM_EPOCHS = 70
INIT_LR = 5e-3 # taxa de aprendizagem inicial

def poly_decay(epoch):

    # Inicializa o número máximo de épocas, a taxa básica de aprendizado e a potência do polinômio
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0

    # Calcula a nova taxa de aprendizagem baseada em polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power

    # Retorna a nova taxa de aprendizagem
    return alpha

# Carrega os dados de treinamento e teste, convertendo as imagens de inteiros para float
print("Carregando o Dataset CIFAR-10...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

# Aplica subtração média aos dados
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

# Converte os labels de inteiros para vetores
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Construindo o Gerador de Imagens para o Data Augmentation
aug = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, fill_mode="nearest")

# Callbacks
figPath = os.path.sep.join([output, "{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output, "{}.json".format(os.getpid())])
callbacks = [TrainingMonitor(figPath, jsonPath=jsonPath), LearningRateScheduler(poly_decay)]

# Inicializa o otimizador e o modelo
print("Compilando o modelo...")
opt = SGD(lr=INIT_LR, momentum=0.9)
model = MiniGoogLeNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# Treina a rede
print("Treinamento...")
model.fit_generator(aug.flow(trainX,
                             trainY,
                             batch_size=64),
                    validation_data=(testX, testY),
                    steps_per_epoch=len(trainX) // 64,
                    epochs=NUM_EPOCHS,
                    callbacks=callbacks,
                    verbose=1)

# Salva o modelo
print("Salvando o modelo...")
modelo_path='minigooglenet_cifar10.hdf5'
model.save(modelo_path)



Using TensorFlow backend.


Carregando o Dataset CIFAR-10...
170500096/170498071 [==============================] - 20s 0us/step
Compilando o modelo...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Treinamento...
Instructions for updating:
Use tf.cast instead.
Epoch 1/70
781/781 [==============================] - 46s 59ms/step - loss: 1.4877 - acc: 0.4587 - val_loss: 2.1116 - val_acc: 0.4352
Epoch 2/70
781/781 [==============================] - 40s 51ms/step - loss: 1.0791 - acc: 0.6154 - val_loss: 1.0169 - val_acc: 0.6456
Epoch 3/70
781/781 [==============================] - 40s 51ms/step - loss: 0.9102 - acc: 0.6806 - val_loss: 1.0003 - val_acc: 0.6487
Epoch 4/70
781/781 [==============================] - 40s 51ms/step - loss: 0.8026 - acc: 0.7199 - val_loss: 0.8339 - val_acc: 0.7166
Epoch 5/70
781/781 [==============================] - 40s 51ms/step - loss: 0.7198 - acc: 

In [4]:
!ls

minigooglenet_cifar10.hdf5  __pycache__  trainingmonitor.py
minigooglenet.py	    sample_data


## Faz download do arquivo de pesos do Modelo

In [0]:
from google.colab import files 
modelo_path='minigooglenet_cifar10.hdf5'
files.download(modelo_path)